In [1]:
import fasttext
model = fasttext.load_model('./util/cc.en.300.bin')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import re

from tqdm import tqdm

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import util.normalization as norm
import util.model as models
import util.utility as util

# Import Data

In [4]:
# Opening JSON file
with open("./mcd_result/map_second_mcd.json") as json_file:
    map_second_keys = json.load(json_file)

with open("./mcd_result/map_category_mcd.json") as json_file:
    map_category = json.load(json_file)

with open("./mcd_result/list_first_cycle_mcd.json") as json_file:
    list_first_cycle = json.load(json_file)

In [5]:
# # Open residu

# df_residu = pd.read_csv("./mcd_result/residu_mcd.csv").fillna("")
# print(df_residu.info())
# df_residu['residu'] = df_residu['residu'].apply(lambda x: str(x).split(', '))

In [6]:
df_base = pd.read_csv("./mcd_result/base_mcd_v1.csv")

df_base['aspect'] = df_base['aspect'].apply(lambda x: x.split(", "))
df_base['token_sentence'] = df_base['token_sentence'].apply(lambda x: x.split(".\n"))
df_base['token_lemma'] = df_base['token_lemma'].apply(lambda x: x.split(".\n"))

df_base['residu'] = df_base['residu'].apply(lambda x: str(x).split(', '))

print(df_base.info())
df_base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewer_id     100 non-null    int64 
 1   review          100 non-null    object
 2   summarize       100 non-null    object
 3   aspect          100 non-null    object
 4   token_sentence  100 non-null    object
 5   token_lemma     100 non-null    object
 6   rating          100 non-null    int64 
 7   residu          100 non-null    object
dtypes: int64(2), object(6)
memory usage: 6.4+ KB
None


,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,residu
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,"[normal transaction, clear substance, someone ...",[Why does it look like someone spit on my food...,"[why do it look like someone spit on my food, ...",1,"[everyone, clear substance, someone spit, norm..."
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"[atmosphere, staff, food, mcdonalds]",[The staff at McDonalds are always friendly an...,[the staff at mcdonalds be always friendly and...,4,[atmosphere]
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,"[mobile order, line, refund, money]","[Never got the refund in the app., Made a mobi...","[never get the refund in the app ., make a mob...",1,"[refund, money, mobile order]"
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"[chicken sandwich, customer service, customer,...",[Crispy chicken sandwich was customer service...,[crispy chicken sandwich be customer service b...,5,[crispy chicken]
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","[order, large meal, english, close attention, ...","[I repeat my order 3 times in the drive thru, ...","[i repeat my order 3 time in the drive thru, i...",1,"[english, close attention, double filet]"


In [7]:
df_nodes = pd.read_csv("./mcd_result/nodes_mcd.csv")
print(df_nodes.info())
df_nodes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   node_id       496 non-null    int64 
 1   reviewer_id   496 non-null    int64 
 2   first_cycle   496 non-null    object
 3   second_cycle  496 non-null    object
 4   category      496 non-null    object
dtypes: int64(2), object(3)
memory usage: 19.5+ KB
None


,node_id,reviewer_id,first_cycle,second_cycle,category
0,0,1,other,drink,food and drink
1,1,1,normal,situation,place and service
2,2,2,mcdonalds,place,place and service
3,3,2,food,food,food and drink
4,4,2,other,place,place and service


# Sentiment Analysis

## A. Sentiment Sentence

In [8]:
df_sentiment = df_base[['reviewer_id', 'token_lemma']].copy().explode('token_lemma')

tqdm.pandas()
df_sentiment['pattern_prediction'] = df_sentiment['token_lemma']\
                                    .progress_apply(models.pattern_lexicon_model)\
                                    .map({'positive': 1,'negative': -1})

df_sentiment = df_sentiment.groupby('reviewer_id')['pattern_prediction'].apply(list).reset_index()
print(df_sentiment.info())
df_sentiment

100%|██████████████████████████████████████████████████████████████████████████████| 297/297 [00:00<00:00, 2036.37it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   reviewer_id         100 non-null    int64 
 1   pattern_prediction  100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB
None


,reviewer_id,pattern_prediction
0,1,"[-1, 1, -1, -1]"
1,2,"[1, 1]"
2,3,"[-1, -1, -1, -1, -1]"
3,4,"[-1, 1]"
4,5,"[-1, 1, -1, -1]"
...,...,...
95,96,"[-1, -1, -1]"
96,97,[-1]
97,98,"[-1, -1, -1]"
98,99,"[1, 1]"


## B. Inverse Coding Process

In [9]:
# Inverse coding process

df_1 = df_base[['reviewer_id', 'aspect', 'token_lemma', 'residu']].copy()
df_2 = df_nodes[['reviewer_id', 'node_id', 'first_cycle', 'second_cycle', 'category']].copy()

df_temp = df_1.merge(df_2, on='reviewer_id', how='inner')
df_temp = df_temp.merge(df_sentiment, on='reviewer_id', how='left')
print(df_temp.info())
df_temp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   reviewer_id         496 non-null    int64 
 1   aspect              496 non-null    object
 2   token_lemma         496 non-null    object
 3   residu              496 non-null    object
 4   node_id             496 non-null    int64 
 5   first_cycle         496 non-null    object
 6   second_cycle        496 non-null    object
 7   category            496 non-null    object
 8   pattern_prediction  496 non-null    object
dtypes: int64(2), object(7)
memory usage: 35.0+ KB
None


,reviewer_id,aspect,token_lemma,residu,node_id,first_cycle,second_cycle,category,pattern_prediction
0,1,"[normal transaction, clear substance, someone ...","[why do it look like someone spit on my food, ...","[everyone, clear substance, someone spit, norm...",0,other,drink,food and drink,"[-1, 1, -1, -1]"
1,1,"[normal transaction, clear substance, someone ...","[why do it look like someone spit on my food, ...","[everyone, clear substance, someone spit, norm...",1,normal,situation,place and service,"[-1, 1, -1, -1]"
2,2,"[atmosphere, staff, food, mcdonalds]",[the staff at mcdonalds be always friendly and...,[atmosphere],2,mcdonalds,place,place and service,"[1, 1]"
3,2,"[atmosphere, staff, food, mcdonalds]",[the staff at mcdonalds be always friendly and...,[atmosphere],3,food,food,food and drink,"[1, 1]"
4,2,"[atmosphere, staff, food, mcdonalds]",[the staff at mcdonalds be always friendly and...,[atmosphere],4,other,place,place and service,"[1, 1]"


In [10]:
def inverse_score(x):
    # Define storage

    keywords = [] # Keywords for pattern search
    keys = [] # Key value for storing
    scoring =  {}
    
    
    # Extract all values
    first = x['first_cycle']
    second = x['second_cycle']
    residu = x['residu']
    aspect = x['aspect']
    tokens = x['token_lemma']
    scores = x['pattern_prediction']
    category = x['category']
    
    
    
    ################### INVERSING PROCESS ##########################################
    # Get the aspect of each code
    if first == 'other':
        for res in residu:
            nearest = util.get_nearest_word(res, [second], model, threshold=.35)
            if nearest:
                keywords.append(res)
                keys.append(second)
                break
    
    else:
        pattern = re.compile(rf"{first}")
        for element in aspect:
            if pattern.search(element):
                keywords.append(element)
                keys.append(first)
                

    # If keys and keywords empty, return None
    if (len(keys) == 0) and (len(keywords) == 0):
        return
    
    ################################# SCORING ##################################################
    # Get the index sentence
    for keyword in keywords:
        pattern = re.compile(rf"{keyword}")
        
        for idx, element in enumerate(tokens):
            if pattern.search(element):
                if keyword not in scoring:
                    scoring[keyword] = scores[idx]
                else:
                    scoring[keyword] = scoring[keyword] + scores[idx]

        if keyword not in scoring:
            # If it's not detected we have to split it and choose the token that contains all element.
            temp_split_result = {}
            for e in keyword.split(' '):
                pattern = re.compile(rf"{e}")
                
                for idx, element in enumerate(tokens):
                    if pattern.search(element):
                        if keyword not in scoring:
                            scoring[keyword] = scores[idx]
                        else:
                            scoring[keyword] = scoring[keyword] + scores[idx]

    # Scoring    
    total_scores = sum(list(scoring.values()))
    tags = '/'.join(keywords)
    
    if total_scores < 0:
        return (second, tags, 'negative')
    else:
        return (second, tags, 'positive')
#     return (second, tags, total_scores)

df_temp['key_score_aspect'] = [inverse_score(x[-1]) for x in df_temp.iterrows()]
# If positive => sentiment 1, if negative => sentiment 0, if empty => sentiment 1 
df_temp['sentiment'] = df_temp['key_score_aspect'].apply(lambda x: 1 if not x else 1 if x[-1] == 'positive' else 0)

# print(inverse_score(df_temp.iloc[495]))
# print(inverse_score(df_temp.iloc[497]))

df_temp

,reviewer_id,aspect,token_lemma,residu,node_id,first_cycle,second_cycle,category,pattern_prediction,key_score_aspect,sentiment
0,1,"[normal transaction, clear substance, someone ...","[why do it look like someone spit on my food, ...","[everyone, clear substance, someone spit, norm...",0,other,drink,food and drink,"[-1, 1, -1, -1]","(drink, someone spit, negative)",0
1,1,"[normal transaction, clear substance, someone ...","[why do it look like someone spit on my food, ...","[everyone, clear substance, someone spit, norm...",1,normal,situation,place and service,"[-1, 1, -1, -1]","(situation, normal transaction, positive)",1
2,2,"[atmosphere, staff, food, mcdonalds]",[the staff at mcdonalds be always friendly and...,[atmosphere],2,mcdonalds,place,place and service,"[1, 1]","(place, mcdonalds, positive)",1
3,2,"[atmosphere, staff, food, mcdonalds]",[the staff at mcdonalds be always friendly and...,[atmosphere],3,food,food,food and drink,"[1, 1]","(food, food, positive)",1
4,2,"[atmosphere, staff, food, mcdonalds]",[the staff at mcdonalds be always friendly and...,[atmosphere],4,other,place,place and service,"[1, 1]","(place, atmosphere, positive)",1
...,...,...,...,...,...,...,...,...,...,...,...
491,99,"[allways, best fry, ice cream]",[allways have the best fry and ice cream in th...,[allways],491,fry,food,food and drink,"[1, 1]","(food, best fry, positive)",1
492,100,"[order, people, english, mcdonalds]","[mcdonalds be great, they really need to hire ...",[english],492,mcdonalds,place,place and service,"[1, -1, 1, -1]","(place, mcdonalds, positive)",1
493,100,"[order, people, english, mcdonalds]","[mcdonalds be great, they really need to hire ...",[english],493,people,situation,place and service,"[1, -1, 1, -1]","(situation, people, positive)",1
494,100,"[order, people, english, mcdonalds]","[mcdonalds be great, they really need to hire ...",[english],494,other,other,other,"[1, -1, 1, -1]",None,1


# Sentiment Scoring

In [11]:
result = df_temp[['node_id', 'reviewer_id', 
                  'first_cycle', 'second_cycle', 
                  'category', 'sentiment']].copy()

result

,node_id,reviewer_id,first_cycle,second_cycle,category,sentiment
0,0,1,other,drink,food and drink,0
1,1,1,normal,situation,place and service,1
2,2,2,mcdonalds,place,place and service,1
3,3,2,food,food,food and drink,1
4,4,2,other,place,place and service,1
...,...,...,...,...,...,...
491,491,99,fry,food,food and drink,1
492,492,100,mcdonalds,place,place and service,1
493,493,100,people,situation,place and service,1
494,494,100,other,other,other,1


In [12]:
first_sentiment = result.groupby('first_cycle')\
                    .agg({'sentiment': lambda x: round(np.sum(x) / len(x), 3)})\
                    .reset_index()\
                    .rename(columns={'sentiment': 'first_cycle_sentiment_positive_rate'})

second_sentiment = result.groupby('second_cycle')\
                    .agg({'sentiment': lambda x: round(np.sum(x) / len(x), 3)})\
                    .reset_index()\
                    .rename(columns={'sentiment': 'second_cycle_sentiment_positive_rate'})

category_sentiment = result.groupby('category')\
                    .agg({'sentiment': lambda x: round(np.sum(x) / len(x), 3)})\
                    .reset_index()\
                    .rename(columns={'sentiment': 'category_sentiment_positive_rate'})


result = result.merge(first_sentiment, on='first_cycle', how='left')
result = result.merge(second_sentiment, on='second_cycle', how='left')
result = result.merge(category_sentiment, on='category', how='left')
result['overall_sentiment_positive_rate'] = round(np.sum(result['sentiment']) / result.shape[0], 3) * np.ones(result.shape[0])


result = result.drop('sentiment', axis=1)

result

,node_id,reviewer_id,first_cycle,second_cycle,category,first_cycle_sentiment_positive_rate,second_cycle_sentiment_positive_rate,category_sentiment_positive_rate,overall_sentiment_positive_rate
0,0,1,other,drink,food and drink,0.574,0.412,0.345,0.425
1,1,1,normal,situation,place and service,1.000,0.348,0.381,0.425
2,2,2,mcdonalds,place,place and service,0.500,0.408,0.381,0.425
3,3,2,food,food,food and drink,0.500,0.324,0.345,0.425
4,4,2,other,place,place and service,0.574,0.408,0.381,0.425
...,...,...,...,...,...,...,...,...,...
491,491,99,fry,food,food and drink,0.455,0.324,0.345,0.425
492,492,100,mcdonalds,place,place and service,0.500,0.408,0.381,0.425
493,493,100,people,situation,place and service,0.400,0.348,0.381,0.425
494,494,100,other,other,other,0.574,0.653,0.653,0.425


# Add Review Value

In [13]:
df_rating = df_base[['reviewer_id', 'rating']]

result = result.merge(df_rating, on='reviewer_id', how='left')

first_rating = result.groupby('first_cycle')\
                .agg({'rating': 'mean'})\
                .apply(lambda x: round(x, 3))\
                .reset_index()\
                .rename(columns={'rating': 'first_cycle_rating'})
second_rating = result.groupby('second_cycle')\
                .agg({'rating': 'mean'})\
                .apply(lambda x: round(x, 3))\
                .reset_index()\
                .rename(columns={'rating': 'second_cycle_rating'})
category_rating = result.groupby('category')\
                .agg({'rating': 'mean'})\
                .apply(lambda x: round(x, 3))\
                .reset_index()\
                .rename(columns={'rating': 'category_rating'})

result = result.merge(first_rating, on='first_cycle', how='left')
result = result.merge(second_rating, on='second_cycle', how='left')
result = result.merge(category_rating, on='category', how='left')
result['overall_rating'] = round(np.sum(result['rating']) / result.shape[0], 3) * np.ones(result.shape[0])

result = result.drop('rating', axis=1)

result

,node_id,reviewer_id,first_cycle,second_cycle,category,first_cycle_sentiment_positive_rate,second_cycle_sentiment_positive_rate,category_sentiment_positive_rate,overall_sentiment_positive_rate,first_cycle_rating,second_cycle_rating,category_rating,overall_rating
0,0,1,other,drink,food and drink,0.574,0.412,0.345,0.425,2.139,1.794,1.993,1.99
1,1,1,normal,situation,place and service,1.000,0.348,0.381,0.425,1.667,1.565,1.940,1.99
2,2,2,mcdonalds,place,place and service,0.500,0.408,0.381,0.425,1.750,2.014,1.940,1.99
3,3,2,food,food,food and drink,0.500,0.324,0.345,0.425,2.700,2.054,1.993,1.99
4,4,2,other,place,place and service,0.574,0.408,0.381,0.425,2.139,2.014,1.940,1.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,491,99,fry,food,food and drink,0.455,0.324,0.345,0.425,1.818,2.054,1.993,1.99
492,492,100,mcdonalds,place,place and service,0.500,0.408,0.381,0.425,1.750,2.014,1.940,1.99
493,493,100,people,situation,place and service,0.400,0.348,0.381,0.425,3.000,1.565,1.940,1.99
494,494,100,other,other,other,0.574,0.653,0.653,0.425,2.139,1.944,1.944,1.99


In [14]:
# Update base

df_temp['key_score_aspect'] = df_temp['key_score_aspect'].apply(lambda x: ", ".join(list(x)) if x else '')
df_int = df_temp[['reviewer_id', 'key_score_aspect']]\
            .groupby('reviewer_id')\
            .agg({'key_score_aspect': lambda x: "; ".join(x)})\
            .reset_index()
df_base = df_base.merge(df_int, on='reviewer_id', how='left')

df_base['aspect'] = df_base['aspect'].apply(lambda x: ", ".join(x))
df_base['token_sentence'] = df_base['token_sentence'].apply(lambda x: ", ".join(x))
df_base['token_lemma'] = df_base['token_lemma'].apply(lambda x: ", ".join(x))
df_base['residu'] = df_base['residu'].apply(lambda x: ", ".join(x))

df_base.head()

,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,residu,key_score_aspect
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,"normal transaction, clear substance, someone s...","Why does it look like someone spit on my food,...","why do it look like someone spit on my food, i...",1,"everyone, clear substance, someone spit, norma...","drink, someone spit, negative; situation, norm..."
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"atmosphere, staff, food, mcdonalds",The staff at McDonalds are always friendly and...,the staff at mcdonalds be always friendly and ...,4,atmosphere,"place, mcdonalds, positive; food, food, positi..."
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,"mobile order, line, refund, money","Never got the refund in the app., Made a mobil...","never get the refund in the app ., make a mobi...",1,"refund, money, mobile order","service, mobile order, negative; situation, li..."
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"chicken sandwich, customer service, customer, ...",Crispy chicken sandwich was customer service ...,crispy chicken sandwich be customer service be...,5,crispy chicken,"service, customer service, positive; food, chi..."
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","order, large meal, english, close attention, l...","I repeat my order 3 times in the drive thru, i...","i repeat my order 3 time in the drive thru, it...",1,"english, close attention, double filet","situation, large meal/large fry, positive; foo..."


In [15]:
df_int

,reviewer_id,key_score_aspect
0,1,"drink, someone spit, negative; situation, norm..."
1,2,"place, mcdonalds, positive; food, food, positi..."
2,3,"service, mobile order, negative; situation, li..."
3,4,"service, customer service, positive; food, chi..."
4,5,"situation, large meal/large fry, positive; foo..."
...,...,...
95,96,"service, time management, negative; service, s..."
96,97,"service, employee, negative; food, order food,..."
97,98,"situation, order wrong, negative; food, order ..."
98,99,"food, ice cream, positive; drink, ice cream, p..."


In [16]:
result.to_csv("./mcd_result/final_nodes.csv", index=False)
df_base.to_csv("./mcd_result/base_mcd_v2.csv", index=False)